In [18]:
#!conda install -c conda-forge tensorflow-gpu --yes

In [19]:
#!conda install -c conda-forge keras --yes

In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
#import cv2
import matplotlib.image as mpimg
%matplotlib inline

In [21]:
#from keras import backend as K
#K.tensorflow_backend._get_available_gpus()

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [22]:
from keras.layers import Dense, Flatten, Activation, MaxPooling2D, Lambda, Dropout
from keras.models import Sequential
from keras.layers.core import Lambda
from keras.layers import Conv2D, Cropping2D 
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

# generator

# read the csv file
driving_df_1 = pd.read_csv('driving_log_1.csv')
driving_df_2A = pd.read_csv('driving_log_2A.csv')

def get_images_filenames_measurements(dataframe):

    filenames = []
    measurements = []
    
    correction = 0.15

    for index, row in dataframe.iterrows():
        center_text = row[0]
        center_i = center_text.find('center')
        center_filename = center_text[center_i:]
        #center_filenames.append(center_filename)
        center_measure = float(row[3])
        filenames.append(center_filename)
        measurements.append(center_measure)
        left_text = row[1]
        left_i = left_text.find('left')
        left_filename = left_text[left_i:]
        #left_filenames.append(left_filename)
        left_measure = center_measure + correction
        #left_measures.append(left_measure)
        filenames.append(left_filename)
        measurements.append(left_measure)
        right_text = row[2]
        right_i = right_text.find('right')
        right_filename = right_text[right_i:]
        #right_filenames.append(right_filename)
        right_measure = center_measure - correction
        filenames.append(right_filename)
        measurements.append(right_measure)
        #right_measures.append(right_measure)
    
        
    return filenames, measurements

filenames_1, measurements_1 = get_images_filenames_measurements(driving_df_1)
filenames_2, measurements_2 = get_images_filenames_measurements(driving_df_2A)

filenames = np.append(np.asarray(filenames_1), np.asarray(filenames_2))
measurements = np.append(np.asarray(measurements_1), np.asarray(measurements_2))

flips_filenames = []

for each in filenames:
    flips_filenames.append('f'+each)
    
flips_measurements = - np.asarray(measurements)

total_filenames = np.append(np.asarray(filenames), np.asarray(flips_filenames))
total_measurements = np.append(np.asarray(measurements), np.asarray(flips_measurements))

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(total_filenames, total_measurements, test_size=0.25, random_state=101)

In [23]:
def generator(X_data, y_data, batch_size):
    
    num_samples = len(X_data)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_x = []
            batch_y = []
            X_data_range, y_data_range = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
            if len(X_data_range) < batch_size:
                break
            for i in range(0, batch_size):
                filename = X_data_range[i]
                current_path = 'IMG6/' + filename
                #print(current_path)
                #batch_x_one = cv2.imread(current_path)
                batch_x_one = mpimg.imread(current_path)
                #print(batch_x_one.shape)
                #if batch_x_one is None:
                #    print("null")
                #plt.imshow(batch_x_one)
                #batch_x_one = cv2.cvtColor(batch_x_one, cv2.COLOR_BGR2GRAY)
                #batch_x_one = np.expand_dims(batch_x_one[:, :], 2)
                batch_x.append(batch_x_one)
                batch_y.append(y_data_range[i])
        #batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
            batch_x = np.asarray(batch_x)
            #batch_x = batch_x.reshape(-1, 160, 320, 3)
            batch_y = np.asarray(batch_y)
            batch_x, batch_y = shuffle(batch_x, batch_y)
            yield batch_x, batch_y
    
batch_x, batch_y = next(generator(X_train, y_train, 32))

batch_size = 64
epochs = 1

train_generator = generator(X_train, y_train, batch_size)
valid_generator = generator(X_valid, y_valid, batch_size)




In [24]:
adam = Adam(lr=0.00003, beta_1=0.9, beta_2=0.999, decay=0.000003)


In [25]:
model = Sequential()

# crop the some of the top and bottom to make image more relevant
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))

# doing the normalizaton
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(90,320,3)))
#model.add(Cropping2D(cropping=((50,20), (0,0))))
# convolution layer 1, with input: x1 output: x24
#model.add(Conv2D(filters=8, kernel_size=(5, 5), padding='same', subsample=(2, 2)))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
# maxpooling layer 1, with input: x24 output: x24
#model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# convolution layer 2, with input: x24, output: x36
model.add(Conv2D(filters=15, kernel_size=(5, 5), padding='same', subsample=(2, 2)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
# maxpooling layer 2, with input: x36 output: x36
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# convolution layer 3, with input: x36, output: x48
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='same', subsample=(2, 2)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
# maxpooling layer 3, with input: x48, output: x48
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# flatten the inputs, prepared for the fully connected layer
model.add(Flatten())
# fully connected layer 1, with input x48, output: 1024
model.add(Dense(500))
#model.add(BatchNormalization())
model.add(Activation('relu'))
# apply regularization, with a dropout probability of 50%
model.add(Dropout(0.5))
# fully connected layer 2, with input 1024, output: 256
model.add(Dense(50))
#model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dense(128))
#model.add(Activation('relu'))
# output layer, with input: 256, output 1
model.add(Dense(1))

model.compile(loss='mse', optimizer=adam, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_2 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 45, 160, 15)       1140      
_________________________________________________________________
activation_5 (Activation)    (None, 45, 160, 15)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 44, 159, 15)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 80, 64)        24064     
_________________________________________________________________
activation_6 (Activation)    (None, 22, 80, 64)        0         
__________

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=15, kernel_size=(5, 5), padding="same", strides=(2, 2))`
  app.launch_new_instance()
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=64, kernel_size=(5, 5), padding="same", strides=(2, 2))`


In [26]:
from keras.callbacks import ModelCheckpoint

# define the checkpoint
filepath = "model91.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [27]:
model.fit_generator(train_generator, 
                    samples_per_epoch=len(X_train), 
                    steps_per_epoch=len(X_train)/batch_size,
                    validation_data=valid_generator,
                    validation_steps=len(X_valid)/batch_size,
                    verbose=1,
                    shuffle=True,
                    nb_epoch=epochs)

model.save('model911.h5')

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=54427, validation_data=<generator..., validation_steps=283.484375, verbose=1, shuffle=True, epochs=1)`
  


Epoch 1/1
54427/54427 [==============================] - 7656s 141ms/step - loss: 0.0100 - acc: 0.2027 - val_loss: 0.0248 - val_acc: 0.2008


In [28]:
# the first saved model is model8.h5

# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
#new_model.fit(x_train, y_train, epochs=5, batch_size=50, callbacks=callbacks_list)
model.fit_generator(train_generator, 
                    samples_per_epoch=len(X_train), 
                    steps_per_epoch=len(X_train)/batch_size,
                    validation_data=valid_generator,
                    validation_steps=len(X_valid)/batch_size,
                    verbose=1,
                    shuffle=True,
                    nb_epoch=epochs,
                    callbacks=callbacks_list)

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=54427, validation_data=<generator..., validation_steps=283.484375, verbose=1, shuffle=True, callbacks=[<keras.ca..., epochs=1)`
  from ipykernel import kernelapp as app


Epoch 1/1
54427/54427 [==============================] - 7639s 140ms/step - loss: 0.0017 - acc: 0.2041 - val_loss: 0.0247 - val_acc: 0.2009

Epoch 00001: loss improved from inf to 0.00172, saving model to model91.h5
